In [1]:
import glob
import os, shutil, sys, time, tqdm
from PIL import Image

from sklearn.model_selection import train_test_split

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

In [2]:
### 準備資料夾路徑

dirs = ['train', 'valid', 'test', 'source']  # source for temp process
classes = ['0', '1', '2', '3', 'pp']   # pp for temp process
for d in dirs:
    try:
        os.mkdir('datasets/{}'.format(d))
    except:
        pass
        
    for c in classes:
        try:
            os.mkdir('datasets/{}/{}'.format(d, c))
        except:
            pass

# try:
#     os.mkdir('datasets/source')
# except:
#     pass

try:
    os.mkdir('datasets/source - bak')
except:
    pass    

In [3]:
## 把檔案從 to_ocr_print_to_img/output 各資料夾中全部複製到  'datasets/source - bak'

data_source_path = '../to_ocr_print_to_img/output/*/*.jpg'
target_path = 'datasets/source - bak'
filelist = glob.glob(data_source_path)
print(len(filelist))

limit = None
for f in tqdm.tqdm(filelist[:limit]):
    shutil.copy(f, target_path)

6067


100%|█████████████████████████████████████████████████████████████████████████████| 6067/6067 [00:16<00:00, 360.37it/s]


# 手動移動檔案到 source/0,1,2,3 裡面
# 全部用程式旋轉到正向，然後搬移到 pp 資料夾中，再做後續流程

In [4]:
filelist = glob.glob('datasets/source/1/*.jpg')    # 原圖為向右，用程式左轉90度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(90, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    
filelist = glob.glob('datasets/source/2/*.jpg')   # 原圖為向下，用程式轉180度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(-180, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    
filelist = glob.glob('datasets/source/3/*.jpg')   # 原圖為向左，用程式右轉90度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(-90, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    
for i in [0, 1, 2, 3]:
    filelist = glob.glob('datasets/source/{}/*.jpg'.format(i))
    for f in tqdm.tqdm(filelist):
        shutil.move(f, 'datasets/source/pp')

0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████| 4552/4552 [00:01<00:00, 3324.60it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1267/1267 [00:01<00:00, 942.63it/s]


# 搬到 source/pp 之後，清除 image exif (exten info) 這要很久

In [5]:
### 搬到 source/pp 之後，清除 image exif (exten info)

def clearExifInfo(photoAddress):
    image = Image.open(photoAddress)
    data = list(image.getdata())
#     print(len(data), image.mode, image.size, photoAddress)
    image_without_exif = Image.new(image.mode, image.size)
    image_without_exif.putdata(data)
    image_without_exif.save(photoAddress)
    return
 
filelist = glob.glob('datasets/source/pp/*.jpg')
for f in tqdm.tqdm(filelist[:]):
    clearExifInfo(f)

100%|██████████████████████████████████████████████████████████████████████████████| 6067/6067 [15:25<00:00,  6.56it/s]


In [6]:
### 搬到 train valid test / pp 裡面
filelist = glob.glob('datasets/source/pp/*.jpg')
print(len(filelist))
train_list, test_list = train_test_split(filelist, test_size=0.2, shuffle=True)
train, valid = train_test_split(train_list, test_size=0.4, shuffle=True)
print('train', len(train))
print('valid', len(valid))
print('test_list', len(test_list))

for f in tqdm.tqdm(train):   
    shutil.copy(f, 'datasets/train/pp')
    
for f in tqdm.tqdm(valid):
    shutil.copy(f, 'datasets/valid/pp')
    
for f in tqdm.tqdm(test_list):
    shutil.copy(f, 'datasets/test/pp')

6067
train 2911
valid 1942
test_list 1214


100%|█████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:02<00:00, 446.51it/s]


# 裁切處理圖片 five corp + resize,  and resize only

In [7]:
def img_preprocess(img, dirs, name):
    '''
    img : img path str
    path : train, valid, test
    name : img path str
    '''
    name = name.split('\\')[-1][:-4]
    img = Image.open(img)
    cs = TF.five_crop(img, 600)
    for i, c in enumerate(cs):
        c = TF.resize(c, 224)
        c.save('datasets/{}/0/{}_{}.jpg'.format(dirs, name, i))
    # img_resize_only
    ss = TF.resize(img, (224,224))
    ss.save('datasets/{}/0/{}_{}.jpg'.format(dirs, name, 's'))    
    
def img_resize_only(img, dirs, name): # for test
    '''
    img : img path str
    path : train, valid, test
    name : img path str
    '''
    name = name.split('\\')[-1][:-4]
    img = Image.open(img)
#     cs = TF.five_crop(img, 600)
#     for i, c in enumerate(cs):
#         c = TF.resize(c, 224)
#         c.save('datasets/{}/0/{}_{}.jpg'.format(dirs, name, i))
    ss = TF.resize(img, (224,224))
    ss.save('datasets/{}/0/{}_{}.jpg'.format(dirs, name, 's'))        
    

In [8]:
dirs = ['train', 'valid', 'test']
count = 0
for d in dirs:
    filelist = glob.glob('datasets/' + d + '/pp/*.jpg')
    if d == 'test':
        for f in tqdm.tqdm(filelist):
            img_resize_only(f, d, f)
            count += 1    
    else:
        for f in tqdm.tqdm(filelist):
            img_preprocess(f, d, f)
            count += 1
        
print(count)


100%|██████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:13<00:00, 93.09it/s]

6067


# 接著從0資料夾旋轉並複製到其他資料夾1,2,3 並刪除 pp 資料夾

In [9]:
dirs = ['train', 'valid', 'test']  # source for temp process

In [10]:
for d in dirs:
    filelist = glob.glob('datasets/{}/0/*.jpg'.format(d))
    for demo_file in tqdm.tqdm(filelist):
        img = Image.open(demo_file)
        img=img.rotate(-90, expand = True)  # -90 right
        img.save('datasets/{}/1/{}'.format(d, demo_file.split('\\')[-1]))    
    for demo_file in tqdm.tqdm(filelist):
        img = Image.open(demo_file)
        img=img.rotate(-180, expand = True)  # -180 upside-down
        img.save('datasets/{}/2/{}'.format(d, demo_file.split('\\')[-1]))   
    for demo_file in tqdm.tqdm(filelist):
        img = Image.open(demo_file)
        img=img.rotate(90, expand = True)  # 90 left
        img.save('datasets/{}/3/{}'.format(d, demo_file.split('\\')[-1])) 
                 
    shutil.rmtree('datasets/{}/pp'.format(d))
    print('rmtree {} pp'.format(d))
    
    
        
        

100%|███████████████████████████████████████████████████████████████████████████| 17466/17466 [00:48<00:00, 361.88it/s]


rmtree train pp


100%|███████████████████████████████████████████████████████████████████████████| 11652/11652 [00:27<00:00, 423.68it/s]


rmtree valid pp


100%|█████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:02<00:00, 462.80it/s]


rmtree test pp


In [ ]:
# ### 搬到 source - bak 之後，清除 image exif (exten info)

# def clearExifInfo(photoAddress):
#     image = Image.open(photoAddress)
#     data = list(image.getdata())
# #     print(len(data), image.mode, image.size, photoAddress)
#     image_without_exif = Image.new(image.mode, image.size)
#     image_without_exif.putdata(data)
#     image_without_exif.save(photoAddress)
#     return
 
# filelist = glob.glob('datasets/source - bak/*.*')
# for f in tqdm.tqdm(filelist[:]):
#     clearExifInfo(f)

In [ ]:
# def correctimg(filelist, target_path):
#     '''
#     旋轉正向並移除標記
#     '''
#     for demo_file in tqdm.tqdm(filelist):
#         img = Image.open(demo_file)
#         try:
#             '''
#             对于手机、相机等设备拍摄的照片，由于手持方向的不同，拍出来的照片可能是旋转0°、90°、180°和270°。
#             即使在电脑上利用软件将其转正，他们的exif信息中还是会保留方位信息。
#             在用PIL读取这些图像时，读取的是原始数据，也就是说，即使电脑屏幕上显示是正常的照片，用PIL读进来后，
#             也可能是旋转的图像，并且图片的size也可能与屏幕上的不一样。
#             对于这种情况，可以利用PIL读取exif中的orientation信息，然后根据这个信息将图片转正后，再进行后续操作，具体如下。
#             '''
#             for orientation in ExifTags.TAGS.keys() : 
#                 if ExifTags.TAGS[orientation]=='Orientation' : break 
#             exif=dict(img._getexif().items())
#             if   exif[orientation] == 3 : 
#                 img=img.rotate(180, expand = True)
#             elif exif[orientation] == 6 : 
#                 img=img.rotate(270, expand = True)
#             elif exif[orientation] == 8 : 
#                 img=img.rotate(90, expand = True)
#         except:
#             pass

#         img.save(target_path + '/' + demo_file.split('\\')[-1])        
#     print('correctimg() job done.')
    
# filelist = glob.glob('datasets/source/pp/*.jpg')
# print(len(filelist))
# target_path = r'datasets/source'
# correctimg(filelist, target_path)